In [2]:
from google.colab import files
uploaded = files.upload()

Saving back_trian_fin_0824.csv to back_trian_fin_0824.csv


In [3]:
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.5/7.5 MB 58.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 29.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 107.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 78.1 MB/s eta 0:00:00


In [1]:
import sys
import numpy as np
import pandas as pd
import random
from tqdm.notebook import tqdm
import os
import re
import time
from collections import Counter

import transformers
from transformers import BertTokenizer,AdamWeightDecay,TFRobertaModel,TFBertModel, AutoModel, AutoTokenizer, AutoModelForMaskedLM, XLMRobertaForSequenceClassification, AdamW
from torch.utils.data import DataLoader, Dataset

import tensorflow as tf
import keras
from keras.callbacks import EarlyStopping,ModelCheckpoint

import sklearn
from sklearn.metrics import confusion_matrix,accuracy_score
from sklearn.model_selection import StratifiedKFold

In [2]:
BASE_DIR = '/content/'

data_path= BASE_DIR + '/data/'
model_path= BASE_DIR + '/model/'
sub_path= BASE_DIR + '/sub/'

In [3]:
# # Load data
train=pd.read_csv(os.path.join(data_path,'train_data.csv'))
test=pd.read_csv(os.path.join(data_path,'test_data.csv'))
sample_submission=pd.read_csv(os.path.join(data_path,'sample_submission.csv'))

In [4]:
train

,index,title,topic_idx
0,0,인천→핀란드 항공기 결항…휴가철 여행객 분통,4
1,1,실리콘밸리 넘어서겠다…구글 15조원 들여 美전역 거점화,4
2,2,이란 외무 긴장완화 해결책은 미국이 경제전쟁 멈추는 것,4
3,3,NYT 클린턴 측근韓기업 특수관계 조명…공과 사 맞물려종합,4
4,4,시진핑 트럼프에 중미 무역협상 조속 타결 희망,4
...,...,...,...
45649,45649,KB금융 미국 IB 스티펠과 제휴…선진국 시장 공략,1
45650,45650,1보 서울시교육청 신종코로나 확산에 개학 연기·휴업 검토,2
45651,45651,게시판 키움증권 2020 키움 영웅전 실전투자대회,1
45652,45652,답변하는 배기동 국립중앙박물관장,2


In [5]:
back_train = pd.read_csv(data_path + '/trans/back_trian_fin_{}.csv'.format('0824'))
back_train

,index,title,topic_idx
0,0,인천→핀란드 항공기 결항…명절 여행객 분노,4
1,1,"실리콘밸리 넘어...구글, 15조원 들여 미국 글로벌 허브로 도약",4
2,2,이란 외무부 긴장완화 해법은 미국이 경제전쟁을 멈추는 것입니다,4
3,3,"NYT 클린턴 한국 기업 최측근, 특수관계 부각…공적·사적 결합",4
4,4,시진핑은 가능한 한 빨리 중미 무역 협상을 타결하기,4
...,...,...,...
45649,45649,"KB금융, 미국 IB 스티펠과 제휴…선진시장 공략",1
45650,45650,"서울시교육청, 코로나19 확산으로 개학 연기·폐교 검토",2
45651,45651,게시판 키움증권 2020 키움영웅대회 실물투자대회,1
45652,45652,배기동 국립중앙박물관장 답변,2


In [6]:
tokenizer = AutoTokenizer.from_pretrained('xlm-roberta-large', force_download = True)

In [7]:
import torch
device = "cuda" if torch.cuda.is_available() else "cpu"
print(device)

cuda


In [8]:
# train, val에 사용
class CompDataset(Dataset):

    def __init__(self, df):
        self.df_data = df

    def __getitem__(self, index):

        # 데이터프레임 칼럼 들고오기
        title = self.df_data.loc[index, 'title']
        # sentence2 = self.df_data.loc[index, 'hypothesis']

        encoded_dict = tokenizer.encode_plus(
                    title,
                    add_special_tokens = True,
                    max_length = MAX_LEN,
                    pad_to_max_length = True,
                    truncation=True,
                    return_attention_mask = True,
                    return_tensors = 'pt',
               )

        padded_token_list = encoded_dict['input_ids'][0]
        att_mask = encoded_dict['attention_mask'][0]

        # 숫자로 변환된 label을 텐서로 변환
        target = torch.tensor(self.df_data.loc[index, 'topic_idx'])
        # input_ids, attention_mask, label을 하나의 인풋으로 묶음
        sample = (padded_token_list, att_mask, target)

        return sample

    def __len__(self):
        return len(self.df_data)

# test 예측에 사용
class TestDataset(Dataset):

    def __init__(self, df):
        self.df_data = df

    def __getitem__(self, index):

        title = self.df_data.loc[index, 'title']


        encoded_dict = tokenizer.encode_plus(
                    title,
                    add_special_tokens = True,
                    max_length = MAX_LEN,
                    pad_to_max_length = True,
                    return_attention_mask = True,
                    truncation=True,
                    return_tensors = 'pt',
               )

        padded_token_list = encoded_dict['input_ids'][0]
        att_mask = encoded_dict['attention_mask'][0]
        # input_ids, attention_mask를 하나의 인풋으로 묶음
        sample = (padded_token_list, att_mask)

        return sample


    def __len__(self):
        return len(self.df_data)

In [9]:
MAX_LEN = 256 # max sequence length
L_RATE = 1e-5 # Controls how large a step is taken when updating model weights during training.
BATCH_SIZE = 16 # The number of examples that will be processed in parallel during training. Tailored for TPUs.
NUM_CORES = os.cpu_count()
NUM_CORES

12

In [10]:
model = XLMRobertaForSequenceClassification.from_pretrained(
    'xlm-roberta-large',
    num_labels = 7
)

model.to(device)

Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at xlm-roberta-large and are newly initialized: ['classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight', 'classifier.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


XLMRobertaForSequenceClassification(
  (roberta): XLMRobertaModel(
    (embeddings): XLMRobertaEmbeddings(
      (word_embeddings): Embedding(250002, 1024, padding_idx=1)
      (position_embeddings): Embedding(514, 1024, padding_idx=1)
      (token_type_embeddings): Embedding(1, 1024)
      (LayerNorm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): XLMRobertaEncoder(
      (layer): ModuleList(
        (0-23): 24 x XLMRobertaLayer(
          (attention): XLMRobertaAttention(
            (self): XLMRobertaSelfAttention(
              (query): Linear(in_features=1024, out_features=1024, bias=True)
              (key): Linear(in_features=1024, out_features=1024, bias=True)
              (value): Linear(in_features=1024, out_features=1024, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): XLMRobertaSelfOutput(
              (dense): Linear(in_features=1024, out_fe

In [11]:
model

XLMRobertaForSequenceClassification(
  (roberta): XLMRobertaModel(
    (embeddings): XLMRobertaEmbeddings(
      (word_embeddings): Embedding(250002, 1024, padding_idx=1)
      (position_embeddings): Embedding(514, 1024, padding_idx=1)
      (token_type_embeddings): Embedding(1, 1024)
      (LayerNorm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): XLMRobertaEncoder(
      (layer): ModuleList(
        (0-23): 24 x XLMRobertaLayer(
          (attention): XLMRobertaAttention(
            (self): XLMRobertaSelfAttention(
              (query): Linear(in_features=1024, out_features=1024, bias=True)
              (key): Linear(in_features=1024, out_features=1024, bias=True)
              (value): Linear(in_features=1024, out_features=1024, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): XLMRobertaSelfOutput(
              (dense): Linear(in_features=1024, out_fe

In [12]:
optimizer = AdamW(model.parameters(), lr = L_RATE, eps = 1e-8)

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [13]:
# Cross validation, StratifiedKfold
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
folds=[]

# Unusual sentences must be included in the training data
# Seperate train set and validation set in each folds
for train_idx, valid_idx in skf.split(train, train['topic_idx']):
    train_idx = np.array(list(set(list(train_idx))))
    valid_idx = np.array(list(set(set(valid_idx))))
    folds.append((train_idx, valid_idx))

In [14]:
train_data_0, val_data_0 = CompDataset(train.iloc[folds[0][0]].append(back_train.iloc[folds[0][0]]).reset_index(drop = True)), CompDataset(train.iloc[folds[0][1]].reset_index(drop = True))
train_data_1, val_data_1 = CompDataset(train.iloc[folds[1][0]].append(back_train.iloc[folds[1][0]]).reset_index(drop = True)), CompDataset(train.iloc[folds[1][1]].reset_index(drop = True))
train_data_2, val_data_2 = CompDataset(train.iloc[folds[2][0]].append(back_train.iloc[folds[2][0]]).reset_index(drop = True)), CompDataset(train.iloc[folds[2][1]].reset_index(drop = True))
train_data_3, val_data_3 = CompDataset(train.iloc[folds[3][0]].append(back_train.iloc[folds[3][0]]).reset_index(drop = True)), CompDataset(train.iloc[folds[3][1]].reset_index(drop = True))
train_data_4, val_data_4 = CompDataset(train.iloc[folds[4][0]].append(back_train.iloc[folds[4][0]]).reset_index(drop = True)), CompDataset(train.iloc[folds[4][1]].reset_index(drop = True))

test_data = TestDataset(test)

# batch_size 만큼 데이터 분할
train_dataloader_0, val_dataloader_0 = DataLoader(train_data_0, batch_size=BATCH_SIZE, shuffle=True, num_workers=NUM_CORES), DataLoader(val_data_0, batch_size=BATCH_SIZE, shuffle=True, num_workers=NUM_CORES)
train_dataloader_1, val_dataloader_1 = DataLoader(train_data_1, batch_size=BATCH_SIZE, shuffle=True, num_workers=NUM_CORES), DataLoader(val_data_1, batch_size=BATCH_SIZE, shuffle=True, num_workers=NUM_CORES)
train_dataloader_2, val_dataloader_2 = DataLoader(train_data_2, batch_size=BATCH_SIZE, shuffle=True, num_workers=NUM_CORES), DataLoader(val_data_2, batch_size=BATCH_SIZE, shuffle=True, num_workers=NUM_CORES)
train_dataloader_3, val_dataloader_3 = DataLoader(train_data_3, batch_size=BATCH_SIZE, shuffle=True, num_workers=NUM_CORES), DataLoader(val_data_3, batch_size=BATCH_SIZE, shuffle=True, num_workers=NUM_CORES)
train_dataloader_4, val_dataloader_4 = DataLoader(train_data_4, batch_size=BATCH_SIZE, shuffle=True, num_workers=NUM_CORES), DataLoader(val_data_4, batch_size=BATCH_SIZE, shuffle=True, num_workers=NUM_CORES)

# val_dataloader = DataLoader(val_data, batch_size=BATCH_SIZE, shuffle=Truem, num_workers=NUM_CORES)

test_dataloader = DataLoader(test_data, batch_size=BATCH_SIZE, shuffle=False, num_workers=NUM_CORES)



print(len(train_dataloader_0))
print(len(val_dataloader_0))
print(len(test_dataloader))

4566
571
571


<ipython-input-14-bde827db4497>:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  train_data_0, val_data_0 = CompDataset(train.iloc[folds[0][0]].append(back_train.iloc[folds[0][0]]).reset_index(drop = True)), CompDataset(train.iloc[folds[0][1]].reset_index(drop = True))
<ipython-input-14-bde827db4497>:2: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  train_data_1, val_data_1 = CompDataset(train.iloc[folds[1][0]].append(back_train.iloc[folds[1][0]]).reset_index(drop = True)), CompDataset(train.iloc[folds[1][1]].reset_index(drop = True))
<ipython-input-14-bde827db4497>:3: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  train_data_2, val_data_2 = CompDataset(train.iloc[folds[2][0]].append(back_train.iloc[folds[2][0]]).reset_index(

In [15]:
# Learn about each fold around the for gate and store the best weights.

NUM_EPOCHS = 5

for k in range(5):
    print('########## Fold {} : \n'.format(k))
    for epoch in range(NUM_EPOCHS):
        print("")
        print('======== Epoch {:} / {:} ========'.format(epoch + 1, NUM_EPOCHS))

        stacked_val_labels = []
        targets_list = []

        print('Training...')

        model.train()
        torch.set_grad_enabled(True)

        total_train_loss = 0

        for i, batch in enumerate(globals()['train_dataloader_{}'.format(k)]):
            train_status = 'Batch ' + str(i) + ' of ' + str(len(globals()['train_dataloader_{}'.format(k)]))

            print(train_status, end='\r')

            b_input_ids = batch[0].to(device)
            b_input_mask = batch[1].to(device)
            b_labels = batch[2].to(device)

            model.zero_grad()

            outputs = model(b_input_ids, attention_mask = b_input_mask, labels = b_labels)
            loss = outputs[0]

            total_train_loss = total_train_loss + loss.item()

            optimizer.zero_grad()
            torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
            optimizer.step()

        print('Train loss: ', total_train_loss)

        print('\nValidation...')

        model.eval()

        torch.set_grad_enabled(False)
        total_val_loss = 0

        for j, batch in enumerate(globals()['val_dataloader_{}'.format(k)]):

            val_status = 'Batch ' + str(j) + ' of ' + str(len(globals()['val_dataloader_{}'.format(k)]))

            print(val_status, end='\r')

            b_input_ids = batch[0].to(device)
            b_input_mask = batch[1].to(device)
            b_labels = batch[2].to(device)


            outputs = model(b_input_ids, attention_mask=b_input_mask, labels=b_labels)

            loss = outputs[0]

            total_val_loss = total_val_loss + loss.item()


            # 예측값
            preds = outputs[1]

            # 예측값을 CPU로 이동시킵니다.
            val_preds = preds.detach().cpu().numpy()

            # labels을 cpu로 이동시킵니다.
            targets_np = b_labels.to('cpu').numpy()

            targets_list.extend(targets_np)

            if j == 0:  # 첫 번째 batch일 떄
                stacked_val_preds = val_preds

            else:
                stacked_val_preds = np.vstack((stacked_val_preds, val_preds))

        # validation accuracy 계산
        y_true = targets_list
        y_pred = np.argmax(stacked_val_preds, axis=1)

        val_acc = accuracy_score(y_true, y_pred)


        print('Val loss:' ,total_val_loss)
        print('Val acc: ', val_acc)


        # 모델 저장
        torch.save(model.state_dict(), 'epoch:{}_model.pt'.format(epoch))

        # 메모리 관리
        gc.collect()

########## Fold 0 : 


======== Epoch 1 / 5 ========
Training...


/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2418: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2418: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input 

KeyboardInterrupt: ignored

In [ ]:
torch.cuda.empty_cache()